In [1]:
pip install certifi

Note: you may need to restart the kernel to use updated packages.


In [2]:
# -------------------------------------------------------------
# train_distill.py  —  Local‑only prototype for adaptive‑β, 
#                      selective‑hint knowledge distillation
# -------------------------------------------------------------
# Works on Linux / macOS / Windows without Google‑Colab APIs.
# Creates ./checkpoints/ for weights, logs, and resumable state.
# ------------------------------------------------------------
import os, ssl, certifi, json, time, math
from pathlib import Path

import torch, torch.nn as nn, torch.nn.functional as F
import torchvision.transforms as T
import torchvision.datasets as dset
import torchvision.models as models
from torch.utils.data import DataLoader
from tqdm.auto import tqdm  # auto handles notebook / terminal
import pandas as pd

# -------------------------------------------------------------
# 0. ENV & PATHS ------------------------------------------------
# -------------------------------------------------------------
# Fix macOS SSL download issues (no harm on Linux/Win)
ssl._create_default_https_context = ssl._create_unverified_context  # noqa: E402

DEVICE       = torch.device("cuda" if torch.cuda.is_available() else "cpu")
CKPT_DIR     = Path("./checkpoints"); CKPT_DIR.mkdir(exist_ok=True)
TEACHER_PATH = CKPT_DIR / "teacher.pth"
STUDENT_PATH = CKPT_DIR / "student.pth"
PROJ_PATH    = CKPT_DIR / "projections.pth"
RESUME_JSON  = CKPT_DIR / "resume.json"
LOG_CSV      = CKPT_DIR / "training_log_adaptive.csv"

print(f"▶ Using device: {DEVICE}")

# -------------------------------------------------------------
# 1. DATA  (CIFAR‑10) ------------------------------------------
# -------------------------------------------------------------
TFMS_TRAIN = T.Compose([
    T.RandomCrop(32, padding=4),
    T.RandomHorizontalFlip(),
    T.ToTensor(),
    T.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])
TFMS_TEST = T.Compose([
    T.ToTensor(),
    T.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])
TRAIN_SET = dset.CIFAR10(root="./data", train=True,  download=True, transform=TFMS_TRAIN)
TEST_SET  = dset.CIFAR10(root="./data", train=False, download=True, transform=TFMS_TEST)
TRAIN_LOADER = DataLoader(TRAIN_SET, batch_size=64, shuffle=True,  num_workers=2, pin_memory=True)
VAL_LOADER   = DataLoader(TEST_SET,  batch_size=64, shuffle=False, num_workers=2, pin_memory=True)

# -------------------------------------------------------------
# 2. TEACHER  ResNet‑18  (optionally fine‑tuned) ---------------
# -------------------------------------------------------------
TEACHER_EPOCHS = 7  # set 0 to skip fine‑tune and keep ImageNet weights

def load_or_train_teacher():
    teacher = models.resnet18(weights=models.ResNet18_Weights.IMAGENET1K_V1)
    teacher.fc = nn.Linear(512, 10)  # CIFAR‑10 head
    teacher = teacher.to(DEVICE)

    if TEACHER_PATH.exists():
        teacher.load_state_dict(torch.load(TEACHER_PATH, map_location=DEVICE))
        print("✅ Loaded saved teacher model.")
        return teacher.eval()

    if TEACHER_EPOCHS == 0:
        print("🚀 Using ImageNet‑pretrained teacher without CIFAR fine‑tune.")
        teacher.eval()
        torch.save(teacher.state_dict(), TEACHER_PATH)
        return teacher

    # quick fine‑tune
    print("🚀 No saved teacher, fine‑tuning from ImageNet weights…")
    opt = torch.optim.SGD(teacher.parameters(), lr=0.01, momentum=0.9, weight_decay=5e-4)
    scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(opt, TEACHER_EPOCHS)
    criterion = nn.CrossEntropyLoss()
    teacher.train()
    for ep in range(1, TEACHER_EPOCHS + 1):
        correct = tot = 0
        for x, y in tqdm(TRAIN_LOADER, desc=f"[Teacher] Epoch {ep}/{TEACHER_EPOCHS}"):
            x, y = x.to(DEVICE), y.to(DEVICE)
            opt.zero_grad()
            out = teacher(x)
            loss = criterion(out, y)
            loss.backward(); opt.step()
            correct += (out.argmax(1) == y).sum().item(); tot += y.size(0)
        acc = 100 * correct / tot
        print(f"[Teacher] Epoch {ep}  Accuracy: {acc:.2f}%")
        scheduler.step()
    torch.save(teacher.state_dict(), TEACHER_PATH)
    teacher.eval()
    return teacher

teacher = load_or_train_teacher()

# -------------------------------------------------------------
# 3.  Select teacher layers by activation variance -------------
# -------------------------------------------------------------
BLOCK_CHANNELS = {"layer1": 64, "layer2": 128, "layer3": 256, "layer4": 512}

@torch.no_grad()
def top_variance_layers(model, loader, k=3, num_batches=10):
    var_sum, cnt = {}, {}
    def hooker(name):
        def _hook(_, __, out):
            v = out.var(dim=[0, 2, 3]).mean().item()
            var_sum[name] = var_sum.get(name, 0.0) + v
            cnt[name] = cnt.get(name, 0) + 1
        return _hook

    hooks = []
    for n, m in model.named_modules():
        if n.startswith("layer") and n.count(".") == 1:  # e.g., layer3.0
            hooks.append(m.register_forward_hook(hooker(n)))
    it = iter(loader)
    for _ in range(num_batches):
        x, _ = next(it)
        model(x.to(DEVICE))
    for h in hooks:
        h.remove()
    avg_var = {k: var_sum[k] / cnt[k] for k in var_sum}
    return sorted(avg_var, key=avg_var.get, reverse=True)[:k]

SELECTED_BLOCKS = top_variance_layers(teacher, TRAIN_LOADER)
print("📌 Selected teacher blocks:", SELECTED_BLOCKS)
TEACHER_CHANNELS = [BLOCK_CHANNELS[b.split('.')[0]] for b in SELECTED_BLOCKS]

# register hooks --------------------------------------------------------
HOOKED = {}

def make_hook(tag):
    def _hook(_, __, out):
        HOOKED[tag] = out.detach()
    return _hook
blocks_dict = dict(teacher.named_modules())
for idx, blk_name in enumerate(SELECTED_BLOCKS):
    blocks_dict[blk_name].register_forward_hook(make_hook(f"t{idx + 1}"))

# -------------------------------------------------------------
# 4.  STUDENT CNN (5 conv layers) ------------------------------
# -------------------------------------------------------------
class StudentCNN(nn.Module):
    def __init__(self, num_classes=10):
        super().__init__()
        self.feats = nn.ModuleList([
            nn.Conv2d(3, 32, 3, padding=1),
            nn.Conv2d(32, 32, 3, padding=1),
            nn.Conv2d(32, 64, 3, stride=2, padding=1),
            nn.Conv2d(64, 64, 3, padding=1),
            nn.Conv2d(64, 128, 3, stride=2, padding=1),
        ])
        self.hint_idx = [1, 3, 4]  # after conv 2, conv 4, conv 5
        self.pool = nn.AdaptiveAvgPool2d(1)
        self.fc = nn.Linear(128, num_classes)
    def forward(self, x):
        feats = {}
        for i, layer in enumerate(self.feats):
            x = F.relu(layer(x))
            if i in self.hint_idx:
                feats[f"s{self.hint_idx.index(i) + 1}"] = x
        out = self.pool(x).view(x.size(0), -1)
        return self.fc(out), feats

STUDENT_CHANNELS = [32, 64, 128]
student = StudentCNN().to(DEVICE)

# Projections -----------------------------------------------------------
class Projections(nn.Module):
    def __init__(self, sch, tch):
        super().__init__()
        self.layers = nn.ModuleList([nn.Conv2d(s, t, 1) for s, t in zip(sch, tch)])
    def forward(self, feat_dict):
        return [proj(feat_dict[f"s{i + 1}"]) for i, proj in enumerate(self.layers)]

projections = Projections(STUDENT_CHANNELS, TEACHER_CHANNELS).to(DEVICE)

print(f"🪶 Student parameters: {sum(p.numel() for p in student.parameters()):,}")

# -------------------------------------------------------------
# 5.  Adaptive‑β KD training -----------------------------------
# -------------------------------------------------------------
T, ALPHA, GAMMA = 4.0, 1.0, 1.0
E1, E2 = 10, 40               # pre‑hint epochs, total epochs
BETA_INIT = 250.0

opt = torch.optim.Adam(list(student.parameters()) + list(projections.parameters()), lr=1e-3)
beta = BETA_INIT
best_kd = float('inf'); stagn = 0; decays = 0; MAX_DECAYS = 3
start_epoch = 0

# ----- resume if possible ------------------------------------
if RESUME_JSON.exists():
    with open(RESUME_JSON) as f:
        ck = json.load(f)
        start_epoch, beta, decays = ck['epoch'], ck['beta'], ck['beta_decay_count']
    student.load_state_dict(torch.load(STUDENT_PATH , map_location=DEVICE))
    projections.load_state_dict(torch.load(PROJ_PATH, map_location=DEVICE))
    print(f"🔁 Resumed at epoch {start_epoch + 1}, β={beta}")

history = []

for epoch in range(start_epoch, E2):
    student.train(); tloss = tkd = thint = correct = 0
    if torch.cuda.is_available():
        torch.cuda.reset_peak_memory_stats()
    for x, y in tqdm(TRAIN_LOADER, desc=f"[Student] Epoch {epoch + 1}/{E2}"):
        x, y = x.to(DEVICE), y.to(DEVICE)
        HOOKED.clear()                               # fresh teacher activations
        with torch.no_grad():
            logits_t = teacher(x)
        logits_s, feats_s = student(x)
        proj_s = projections(feats_s)

        # Hint loss
        hint = 0
        for i in range(3):
            t_feat = HOOKED[f"t{i + 1}"]
            s_feat = proj_s[i]
            if s_feat.shape[-2:] != t_feat.shape[-2:]:
                t_feat = F.interpolate(t_feat, size=s_feat.shape[-2:], mode='bilinear', align_corners=False)
            hint += F.mse_loss(s_feat, t_feat)

        # KD + CE
        kd = F.kl_div(
            F.log_softmax(logits_s / T, 1),
            F.softmax(logits_t / T, 1), reduction='batchmean') * (T ** 2)
        ce = F.cross_entropy(logits_s, y)
        loss = beta * hint if epoch < E1 else ALPHA * kd + beta * hint + GAMMA * ce

        opt.zero_grad(); loss.backward(); opt.step()
        tloss += loss.item(); tkd += kd.item(); thint += hint.item()
        correct += (logits_s.argmax(1) == y).sum().item()

    acc = 100 * correct / len(TRAIN_LOADER.dataset)
    avg_kd = tkd / len(TRAIN_LOADER)
    avg_hint = thint / len(TRAIN_LOADER)
    vram = (torch.cuda.max_memory_allocated() / 1024 ** 3) if torch.cuda.is_available() else 0
    print(f"Epoch {epoch + 1}  Acc {acc:.2f}%  KD {avg_kd:.4f}  β {beta}")

    history.append({"epoch": epoch + 1, "acc": acc, "kd": avg_kd, "hint": avg_hint, "beta": beta, "vram": vram})

    # ---- adaptive β ----------------------------------------------------
    if epoch >= E1:
        if avg_kd + 1e-3 < best_kd:
            best_kd = avg_kd; stagn = 0
        else:
            stagn += 1
        if stagn >= 2 and decays < MAX_DECAYS:
            beta /= 2; decays += 1; stagn = 0
            print(f"🪄 β halved → {beta}")

    torch.save(student.state_dict(), STUDENT_PATH)
    torch.save(projections.state_dict(), PROJ_PATH)
    with open(RESUME_JSON, "w") as f:
        json.dump({"epoch": epoch + 1, "beta": beta, "beta_decay_count": decays}, f)

# -------------------------------------------------------------
# 6.  Save training log ---------------------------------------
# -------------------------------------------------------------
pd.DataFrame(history).to_csv(LOG_CSV, index=False)
print("✅ Training complete. Log saved to", LOG_CSV)


/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


▶ Using device: cpu
🚀 No saved teacher, fine‑tuning from ImageNet weights…


[Teacher] Epoch 1/7:   0%|          | 0/782 [00:00<?, ?it/s]/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/torch/utils/data/dataloader.py:684: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, then device pinned memory won't be used.
  warnings.warn(warn_msg)
[Teacher] Epoch 1/7: 100%|██████████| 782/782 [03:27<00:00,  3.76it/s]


[Teacher] Epoch 1  Accuracy: 58.81%


[Teacher] Epoch 2/7: 100%|██████████| 782/782 [03:31<00:00,  3.69it/s]


...

Epoch 38  Acc 55.33%  KD 2.9600  β 250.0


[Student] Epoch 39/40: 100%|██████████| 782/782 [04:39<00:00,  2.80it/s]


Epoch 39  Acc 55.77%  KD 2.9301  β 250.0


[Student] Epoch 40/40: 100%|██████████| 782/782 [04:41<00:00,  2.78it/s]


Epoch 40  Acc 55.75%  KD 2.9222  β 250.0
✅ Training complete. Log saved to checkpoints/training_log_adaptive.csv
